## Introduction to TensorBoard 

__TensorBoard__ is a visualization software that comes with any standard TensorFlow installation. In Google’s words: “The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard.”

TensorFlow programs can range from  very simple to super complex problems (using thousands of computations), and they all have two basic components, Operations and Tensors. As explained in the previous tutorials, the idea is that you create a model that consists of a set of operations, feed data in to the model and the tensors will flow between the operations until you get an output tensor, your result.

When fully configures, TensorBoard window will look something like this:

<img src="files/files/3_1.png" width="500" height="1000" >

___Fig1. ___ TensorBoard appearance


TensorBoard was created as a way to help you understand the flow of tensors in your model so that you can debug and optimize it. It is generally used for two main purposes:

1. Graph Visualization
2. Writing Summaries (or Visualizing Learning)

We'll cover this two main usages of TensorBoard in this tutorial. Learning to use TensorBoard early and often will make working with TensorFlow much more enjoyable and productive.

## 1. Gragh Visualizing

Will powerful, TensorFlow computation graphs can become extremely complicated. Visualizing the graph can help you understand and debug it. Here's an example of the visualization at work from TensorFlow website.

<img src="files/files/3_2.gif" width="500" height="1000" >

___Fig2. ___ Visualization of a TensorFlow graph

To make our TensorFlow program __TensorBoard-activated__, we need to add a very few lines of code to it. This will export the TensorFlow operations into a file, called __event file__ (or event log file). TensorBoard is able to read this file and give insight into the model graph and its performance.

Now let's write a simple TensorFlow program and visualize its computation graph with TensorBoard.

### Example 1:
Let's create two constants and add them together. Constant tensors can be defined simply by defining their value:


In [3]:
import tensorflow as tf

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

5


To visualize the program with TensorBoard, we need to write log files of the program. To write event files, we first need to create a __writer__ for those logs, using this code:

In [ ]:
writer = tf.summary.FileWriter([logdir], [graph])

where __[logdir]__ is the folder where you want to store those log files. You can choose [logdir] to be something meaningful such as './graphs'. The second argument __[graph]__ is the graph of the program we're working on. There are two ways to get the graph:
1. Call the graph using __tf.get_default_graph()__, which returns the default graph of the program
2. set it as __sess.graph__ which returns the session's graph (note that this requires us to already have created a session).

We'll show both ways in the following example; however, the second way is more common. Either way, make sure to create a writer only after you’ve defined your graph. Otherwise, the graph visualized on TensorBoard would be incomplete.

Let's add the writer to the first example and visualize the graph.


In [ ]:
import tensorflow as tf

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)

# creating the writer out of the session
# writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

# launch the graph in a session
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(c))
    # don't forget to close the writer at the end
    writer.close()

Now if you run this code, it creates a directory inside your current directory (beside your Python) code which contains the __event file__.

<img src="files/files/3_3.png" width="300" height="600" >

___Fig3. ___ Created directory which contains the event file


Next, go to Terminal and make sure that the present working directory is the same as where you ran your Python code. For example, here we can switch to the directory using 

cd ~/Desktop/tensorboard

Then run:

tensorboard --logdir="./graphs" --port 6006

It will generate a link for you. ctrl+left click on that link (or simply copy it into your browser or just open your browser and go to http://localhost:6006/. This will show the TensorBoard page which will look like:

<img src="files/files/3_4.png" width="500" height="1000" >

___Fig4. ___ TensorBoard page visualizing the graph generated in Example 1




# Create a variable.
w = tf.Variable(<initial-value>, name=<optional-name>)

Some examples of creating scalar and matrix variables are as follows:

In [6]:
s = tf.Variable(2, name="scalar") 
m = tf.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

Variable __W__ defined above will create a matrix with 784 rows and 10 columns which will be initialized by zeros. This can be used as a weight matrix of a feed-forward neural network (or even in a linear regression model) from a layer with 784 neuron to a layer with 10 neuron. We'll see more of this later in this turorial.

__*Note:__ we use tf.Variable() with uppercase "V", and tf.constant with lowercase "c". You don't necessarily need to know the reason, but it's simply because tf.constant is an op, while tf.Variable is a class with multiple ops.

__* IMPORTANT Note:__ Calling tf.Variable to create a variable is the old way of creating a variable. TensorFlow recommends to use the wraper __tf.get_variable__ which accepts the name, shape, etc as its arguments as follows:


In [ ]:
tf.get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None,
                constraint=None)

Some examples are as follows:

In [7]:
s = tf.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("weight_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

### 2.2. Initialize Variables
Variables need to be initialized befor being used. To do so, we have to invoke a __variable initializer operation__ and run the operation on the session. This is the easiest way to initialize variables which initializes all variables at once.

The following toy example shows how we can add an op to initialize the variables.

### Example 3:
create two variables and add them together. Then print out their value and the summation result.

In [ ]:
a = tf.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.add(a, b, name="Add1")

# launch the graph in a session
with tf.Session() as sess:
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

__FailedPreconditionError: Attempting to use uninitialized value__

As you can see, we ran into __FailedPreconditionError: Attempting to use uninitialized value__. This is because we tried to evaluate the variables before initializing them. Let's correct the code by first initializing all variables and then evaluating them.

In [8]:
# create graph
a = tf.get_variable(name="A", initializer=tf.constant(2))
b = tf.get_variable(name="B", initializer=tf.constant(3))
c = tf.add(a, b, name="Add")
# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer operation
    sess.run(init_op)
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

2
3
5


Let's take a quick look at the graph and generated variables:
<img src="files/files/2_3.png" width="1000" height="2000" >
___Fig3. ___ generated graph (Left) and variables (Right)

As you can see, two blue boxes are the generated variables (compare them with constant nodes in Fig. 2) which are added together using "Add" operation.


__*Note:__ Variables are usually used for weights and biases in neural networks.

- __weights__ are usually initialized from a normal distribution using `tf.truncated_normal_initializer()`.

- __biases__ are usually initialized from zeros using `tf.zeros_initializer()`.

Let's look at a very simple example of creating weight and bias variables with proper initialization:

### Example 4:
Create the weight and bias matrices for a fully-connected layer with 2 neuron to another layer with 3 neuron.
In this scenario, the weight and bias variables must be of size $[2, 3]$ and 3 respectively.

In [9]:
# create graph
weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
biases = tf.get_variable(name="b", shape=[3], initializer=tf.zeros_initializer())

# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer
    sess.run(init_op)
    # now we can run our operations
    W, b = sess.run([weights, biases])
    print('weights = {}'.format(W))
    print('biases = {}'.format(b))

weights = [[-0.00376599 -0.00506956  0.00082394]
 [ 0.0016487   0.00981423 -0.00226094]]
biases = [0. 0. 0.]


## 3. Placeholder:
Placeholders are nodes whose value is fed in at execution time. If you have inputs to your network that depend on some external data and you don't want your graph to depend on any real value, placeholders are the datatype you need. In fact, you can build the graph without needing the data. Therefore, they don't need any initial value; only a datatype (such as float32) and a tensor shape so the graph still knows what to compute even though it doesn't have any stored values yet.

Some examples of creating placeholders are as follows:

In [ ]:
a = tf.placeholder(tf.float32, shape=[5])
b = tf.placeholder(dtype=tf.float32, shape=None, name=None)
X = tf.placeholder(tf.float32, shape=[None, 784], name='input')
Y = tf.placeholder(tf.float32, shape=[None, 10], name='label') 

Let's run a simple example.

### Example 5:
Create a constant vector and a placeholder and add them together.

In [ ]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
      print(sess.run(c))

__InvalidArgumentError: You must feed a value for placeholder tensor 'B' with dtype float and shape  $[3]$ error__

As you can see, running this code will run into an error. As you might have guessed, it is simply because the placeholder is empty and there is no way to add an empty tensor to a constant tensor. To solve this, we need to feed the input value to tensor "a". It can be done by creating a dictionary ("d" in the following code) whose key(s) are the placeholders and their values are the desired value to be passed to the placeholder(s), and feeding it to an argument called "feed_dict". In our example, say we want to pass $[1, 2, 3]$ to the placeholder; the code needs to be modified as:


In [10]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
    # create a dictionary:
    d = {b: [1, 2, 3]}
    # feed it to the placeholder
    print(sess.run(c, feed_dict=d)) 

[6. 7. 8.]


The generated graph and variables are as follows:
<img src="files/files/2_4.png" width="700" height="1400" >
___Fig4. ___ generated graph (Left) and variables (Right)



So far so good?


## Creating a Toy Neural Network
Now, we have all the required materials to start building a toy feed-forward neural network with one hidden layer with 200 hidden units (neurons). The computational graph in Tensorflow will look like this:

<img src="files/files/2_5.png" width="300" height="600" >
___Fig5. ___ Schematic of the graph for one layer of the neural network

How many operations (or nodes) you see in this graph? Six, right? The three circles (X, W, b) and three rectangles. We'll go through them one by one and will discuss what is the best way to implement it.

Let's start with the input, X. This can be an input of any type, such as images, signals, etc. The general approach is to feed all inputs to the network and train the trainable parameters (here, W and b) by backpropagating the error signal. Ideally, you need to feed all inputs together, compute the error, and update the parameters. This process is called "Gradient Descent".

*Side Note: In real-world problems, we have thousands and millions of inputs which makes gradient descent computationally expensive. That's why we split the input set into several shorter pieces (called mini-batch) of size B (called mini-batch size) inputs, and feed them one by one. This is called "Stochastic Gradient Descent". The process of feeding each mini-batch of size B to the network, back-propagating errors, and updating the parameters (weights and biases) is called an iteration.

We generally use Placeholders for inputs so that we can build the graph without depending on any real value. The only point is that you need to choose the proper size for the input. Here, we have a feed-forward neural network, and let's assume inputs of size 784 (similar to 28x28 images of MNIST data). The input placeholder can be written as:

In [ ]:
# create the input placeholder
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")

You might wonder what is shape=$[None, 784]$?!

Well, that's the tricky part! Read the above side note one more time. We need to feed B images of size 784 to the network in each training iteration. So the placeholder needs to be of shape=$[B, 784]$. Defining the placeholder shape as $[None, 784]$ means that you can feed any number of images of size 784 (not B images necessarily). This is especially helpful in the evaluation time where you need to feed all validation or test images to the network and compute the performance on all of them.

Enough with the placeholder. Let's continue with the network parameters, W, and b. As explained in the Variable section above, they have to be defined as variables. Since in Tensorflow, gradient updates will be applied to the graph variables, by default. As mentioned, variables need to be initialized.

*Note: Generally, weights (W) are initialized randomly, in it's the simplest form from a normal distribution, say normal distribution with zero mean and standard deviation of 0.01. Biases (b) can be initialized as small constant values, such as 0.

Since the input dimension is 784 and we have 200 hidden units, the weight matrix will be of size $[784, 200]$. We also need 200 biases, one for each hidden unit. The code will be like:

In [ ]:
# create weight matrix initialized randomely from N(0, 0.01)
weight_initer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
W = tf.get_variable(name="Weight", dtype=tf.float32, shape=[784, 200], initializer=weight_initer)

# create bias vector of size 200, all initialized as zero
bias_initer =tf.constant(0., shape=[200], dtype=tf.float32)
b = tf.get_variable(name="Bias", dtype=tf.float32, initializer=bias_initer)

Now let's move on to the rectangle operations. We must multiply input X$_{[None, 784]}$ and weight matrix W$_{[784, 200]}$ which gives a tensor of size $[None, 200]$, then add the bias vector b$_{[200]}$ and eventually pass the final tensor from a ReLU non-linearity:

In [ ]:
# create MatMul node
x_w = tf.matmul(X, W, name="MatMul")
# create Add node
x_w_b = tf.add(x_w, b, name="Add")
# create ReLU node
h = tf.nn.relu(x_w_b, name="ReLU")
 

Okay, we are all set. The created graph looks like this:
<img src="files/files/2_6.png" width="400" height="800" >
___Fig6. ___ Data flow graph of the neural network created in Tensorflow

But how can you visualize this graph? How did you create this figure?! That's the magic of __Tensorboard__. It's thoroughly explained in our next article.

Before closing it, let's run a session on this graph (using 100 images generated by random pixel values) and get the output of hidden units (h). The whole code will be like this:


In [ ]:
# import the tensorflow library
import tensorflow as tf
import numpy as np

# create the input placeholder
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
weight_initer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)

# create network parameters
W = tf.get_variable(name="Weight", dtype=tf.float32, shape=[784, 200], initializer=weight_initer)
bias_initer =tf.constant(0., shape=[200], dtype=tf.float32)
b = tf.get_variable(name="Bias", dtype=tf.float32, initializer=bias_initer)

# create MatMul node
x_w = tf.matmul(X, W, name="MatMul")
# create Add node
x_w_b = tf.add(x_w, b, name="Add")
# create ReLU node
h = tf.nn.relu(x_w_b, name="ReLU") 

# Add an Op to initialize variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # initialize variables
    sess.run(init_op)
    # create the dictionary:
    d = {X: np.random.rand(100, 784)}
    # feed it to placeholder a via the dict 
    print(sess.run(h, feed_dict=d))

Running this code will print out h$_{[100, 200]}$ which are the outputs of 200 hidden units in response to 100 images; i.e. 200 features extracted from 100 images.

We'll continue constructing the loss function and creating the optimizer operations in the next articles. However, we need to learn Tensorboard first to use its amazing features in our neural network code.

Thanks for reading! If you have any question or doubt, feel free to leave a comment below. You can also send us feedback through contact us page.